In [1]:
import numpy as np
import pandas as pd
import requests
from rates_processing import calc_discount
from cds_processing import calc_cds_monthly

In [2]:
start_date = '2001-01-02'
end_date = '2024-01-31'
loss_given_default =0.6

quarterly_discount = calc_discount(start_date, end_date)
cds_spread_mean = calc_cds_monthly('mean')

/Users/Nidhi/Documents/GitHub/P15_DANK/src/interest_rates.py:65: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  processed_df.index = pd.to_datetime(processed_df.index)


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [19]:
lambda_df = 4 * np.log(1+(cds_spread_mean/(4*loss_given_default)))
quarters = range(1, 21)  # 1 to 20 quarters

risky_duration = pd.DataFrame(index=lambda_df.index, columns=lambda_df.columns)
for col in lambda_df.columns:
    quarterly_survival_probability = pd.DataFrame(index=lambda_df.index, columns=quarters)
    for quarter in quarters:
        quarterly_survival_probability[quarter] = np.exp(-((quarter * lambda_df[col]) / 4))
    temp_df = quarterly_survival_probability * quarterly_discount
    risky_duration[col] = 0.25 * temp_df.sum(axis=1)

In [25]:
risky_duration_shifted = risky_duration.shift(1)
cds_spread_mean_shifted = cds_spread_mean.shift(1)
cds_spread_change = cds_spread_mean.diff()

In [41]:
cds_return = ((cds_spread_mean_shifted/ 12) + (cds_spread_change * risky_duration_shifted))

In [42]:
cds_return

,cds_1,cds_2,cds_3,cds_4,cds_5,cds_6,cds_7,cds_8,cds_9,cds_10,cds_11,cds_12,cds_13,cds_14,cds_15,cds_16,cds_17,cds_18,cds_19,cds_20
Date,,,,,,,,,,,,,,,,,,,,
2001-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-02-28,0.000199,0.000260,0.000331,0.000365,0.000429,0.000480,0.000513,0.000574,0.000652,0.000729,0.000817,0.000894,0.000988,0.001237,0.001343,0.001451,0.001586,0.002089,0.003182,0.009403
2001-03-31,0.000199,0.000266,0.000311,0.000341,0.000387,0.000446,0.000512,0.000586,0.000647,0.000685,0.000741,0.000809,0.000898,0.000972,0.001195,0.001397,0.001542,0.001830,0.002409,0.006627
2001-04-30,0.000200,0.000265,0.000310,0.000344,0.000377,0.000414,0.000476,0.000596,0.000639,0.000683,0.000745,0.000793,0.000866,0.001142,0.001243,0.001467,0.001899,0.002328,0.002982,0.006921
2001-05-31,0.000185,0.000241,0.000276,0.000314,0.000356,0.000397,0.000470,0.000506,0.000563,0.000608,0.000650,0.000751,0.000892,0.000958,0.001125,0.001391,0.001576,0.002023,0.002531,0.004658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,0.000150,0.000235,0.000309,0.000356,0.000399,0.000447,0.000512,0.000566,0.000625,0.000685,0.000752,0.000832,0.000969,0.001158,0.001387,0.001649,0.002040,0.002610,0.003731,0.015387
2023-09-30,0.000155,0.000253,0.000324,0.000378,0.000426,0.000481,0.000537,0.000604,0.000665,0.000734,0.000810,0.000884,0.001001,0.001243,0.001482,0.001763,0.002175,0.002824,0.003990,0.016764
2023-10-31,0.000155,0.000263,0.000337,0.000390,0.000445,0.000499,0.000552,0.000618,0.000689,0.000764,0.000844,0.000919,0.001031,0.001269,0.001529,0.001795,0.002223,0.002934,0.004158,0.020464


In [34]:
actual_return = pd.read_csv('/Users/Nidhi/Downloads/He_Kelly_Manela_Factors_And_Test_Assets_monthly.csv')


In [35]:
actual_return = actual_return[['yyyymm','CDS_01','CDS_02','CDS_03','CDS_04','CDS_05','CDS_06','CDS_07','CDS_08','CDS_09','CDS_10','CDS_11','CDS_12','CDS_13','CDS_14','CDS_15','CDS_16','CDS_17','CDS_18','CDS_19','CDS_20']]

In [36]:
actual_return = actual_return.dropna(axis=0)
actual_return

,yyyymm,CDS_01,CDS_02,CDS_03,CDS_04,CDS_05,CDS_06,CDS_07,CDS_08,CDS_09,...,CDS_11,CDS_12,CDS_13,CDS_14,CDS_15,CDS_16,CDS_17,CDS_18,CDS_19,CDS_20
373,200102.0,0.0039,0.0040,0.0033,0.0024,0.0093,0.0030,0.0020,0.0048,0.0044,...,0.0078,-0.0036,0.0042,0.0057,0.0027,-0.0153,-0.0032,0.0070,0.0005,0.0193
374,200103.0,0.0012,0.0041,0.0061,0.0045,0.0073,0.0059,0.0079,0.0071,0.0087,...,0.0060,0.0042,0.0032,0.0068,-0.0012,0.0093,0.0083,-0.0012,0.0072,0.0219
375,200104.0,0.0033,0.0044,0.0044,0.0060,0.0054,0.0037,0.0044,0.0044,0.0068,...,0.0098,0.0106,0.0084,0.0098,0.0083,0.0124,0.0061,0.0109,0.0041,-0.0018
376,200105.0,0.0038,0.0043,0.0045,0.0039,0.0044,0.0039,0.0040,0.0075,0.0045,...,0.0076,0.0078,0.0049,0.0117,0.0108,0.0061,0.0150,0.0152,0.0184,0.0140
377,200106.0,0.0030,0.0027,0.0044,0.0037,0.0079,0.0042,0.0053,0.0035,0.0040,...,0.0031,0.0029,0.0038,0.0045,0.0034,0.0044,0.0020,0.0073,0.0065,-0.0007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,201208.0,0.0005,0.0009,0.0013,0.0011,0.0022,0.0022,0.0024,0.0017,0.0015,...,0.0031,0.0015,0.0035,0.0032,0.0043,0.0062,0.0084,0.0060,0.0051,0.0152
512,201209.0,0.0012,0.0010,0.0020,0.0022,0.0030,0.0023,0.0030,0.0033,0.0039,...,0.0049,0.0059,0.0057,0.0087,0.0070,0.0083,0.0092,0.0120,0.0119,0.0174
513,201210.0,0.0010,0.0009,0.0009,0.0016,0.0018,0.0019,0.0014,0.0010,0.0019,...,0.0008,0.0028,0.0043,0.0042,0.0013,0.0045,0.0033,0.0028,-0.0010,0.0045
514,201211.0,-0.0006,-0.0007,-0.0009,-0.0007,-0.0010,-0.0010,-0.0009,-0.0016,-0.0016,...,-0.0037,-0.0005,-0.0012,0.0005,0.0015,-0.0002,0.0026,0.0027,0.0107,0.0015
